# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [72]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
# !conda install -c conda-forge folium=0.5.0 --yes 
import folium
from bs4 import BeautifulSoup
import requests
import re
print('Library Import Complete')

Library Import Complete


## Question 1

To create the dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

### Fetching the table from website

In [74]:
# The original link does not have the desired format, so changed the link link accordingly 

toronto_data = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.')[0]
toronto_data.head()
# toronto_data

Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront

### Removing 'Not Assigned' entries

In [75]:
toronto_data = toronto_data[toronto_data['Borough']!='Not assigned']
toronto_data.reset_index()
toronto_data.head()
# toronto_data

Postcode           Borough     Neighbourhood
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M6A        North York  Lawrence Heights
6      M6A        North York    Lawrence Manor

### Combining the Entries

In [76]:
toronto_data = toronto_data.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x)).reset_index()
toronto_data.head()
# toronto_data

Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae

In [57]:
toronto_data.shape

(103, 3)

## Question 2

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:


Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

### The latitude and longitude values of Toronto.

In [78]:
df_location = pd.read_csv('https://cocl.us/Geospatial_data')
df_location.head()
# df_location

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

### Merging the Tables

In [79]:
df_location.rename(columns={"Postal Code" : "Postcode"},inplace=True)
toronto_data = pd.merge(toronto_data,df_location,how='inner',on=['Postcode'])
toronto_data.head()
# toronto_data

Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

In [63]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0])
    )

The dataframe has 10 boroughs and 103 neighborhoods.


## Question 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together.

### Finding Geographical Coordinates of Toronto

In [64]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent = 'foursquare')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Geographical coordinates of Toronto is {},{}.'.format(latitude,longitude))

Geographical coordinates of Toronto is 43.653963,-79.387207.


### Creating a map of Toronto with neighborhoods superimposed on top.

In [65]:
map_toronto = folium.Map(location = [latitude,longitude], zoom_start= 10)

for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{},{}'.format(neighbourhood,borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
 
map_toronto

Simplifying the map and only Clustering areas located in Toronto

In [67]:
columns = []
for col in toronto_data['Borough']:
    if re.search('.+Toronto',col):
        columns.append(col)
columns

['East Toronto',
 'East Toronto',
 'East Toronto',
 'East Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'West Toronto',
 'West Toronto',
 'West Toronto',
 'West Toronto',
 'West Toronto',
 'West Toronto',
 'Downtown Toronto',
 'East Toronto']

In [80]:
toronto_data = toronto_data[toronto_data['Borough'].isin(columns)].reset_index()
# toronto_data.head()
toronto_data

index Postcode           Borough  \
0      37      M4E      East Toronto   
1      41      M4K      East Toronto   
2      42      M4L      East Toronto   
3      43      M4M      East Toronto   
4      44      M4N   Central Toronto   
5      45      M4P   Central Toronto   
6      46      M4R   Central Toronto   
7      47      M4S   Central Toronto   
8      48      M4T   Central Toronto   
9      49      M4V   Central Toronto   
10     50      M4W  Downtown Toronto   
11     51      M4X  Downtown Toronto   
12     52      M4Y  Downtown Toronto   
13     53      M5A  Downtown Toronto   
14     54      M5B  Downtown Toronto   
15     55      M5C  Downtown Toronto   
16     56      M5E  Downtown Toronto   
17     57      M5G  Downtown Toronto   
18     58      M5H  Downtown Toronto   
19     59      M5J  Downtown Toronto   
20     60      M5K  Downtown Toronto   
21     61      M5L  Downtown Toronto   
22     63      M5N   Central Toronto   
23     64      M5P   Central Toronto   
24     65      M5R   Central Toronto   
25     66      M5S  Downtown Toronto   
26     67      M5T  Downtown Toronto   
27     68      M5V  Downtown Toronto   
28     69      M5W  Downtown Toronto   
29     70      M5X  Downtown Toronto   
30     75      M6G  Downtown Toronto   
31     76      M6H      West Toronto   
32     77      M6J      West Toronto   
33     78      M6K      West Toronto   
34     82      M6P      West Toronto   
35     83      M6R      West Toronto   
36     84      M6S      West Toronto   
37     85      M7A  Downtown Toronto   
38     87      M7Y      East Toronto   

                                        Neighbourhood   Latitude  Longitude  
0                                         The Beaches  43.676357 -79.293031  
1                        The Danforth West, Riverdale  43.679557 -79.352188  
2                      The Beaches West, India Bazaar  43.668999 -79.315572  
3                                     Studio District  43.659526 -79.340923  
4                                       Lawrence Park  43.728020 -79.388790  
5                                    Davisville North  43.712751 -79.390197  
6                                  North Toronto West  43.715383 -79.405678  
7                                          Davisville  43.704324 -79.388790  
8                         Moore Park, Summerhill East  43.689574 -79.383160  
9   Deer Park, Forest Hill SE, Rathnelly, South Hi...  43.686412 -79.400049  
10                                           Rosedale  43.679563 -79.377529  
11                        Cabbagetown, St. James Town  43.667967 -79.367675  
12                               Church and Wellesley  43.665860 -79.383160  
13                                       Harbourfront  43.654260 -79.360636  
14                           Ryerson, Garden District  43.657162 -79.378937  
15                                     St. James Town  43.651494 -79.375418  
16                                        Berczy Park  43.644771 -79.373306  
17                                 Central Bay Street  43.657952 -79.387383  
18                           Adelaide, King, Richmond  43.650571 -79.384568  
19  Harbourfront East, Toronto Islands, Union Station  43.640816 -79.381752  
20           Design Exchange, Toronto Dominion Centre  43.647177 -79.381576  
21                     Commerce Court, Victoria Hotel  43.648198 -79.379817  
22                                           Roselawn  43.711695 -79.416936  
23                Forest Hill North, Forest Hill West  43.696948 -79.411307  
24                The Annex, North Midtown, Yorkville  43.672710 -79.405678  
25                     Harbord, University of Toronto  43.662696 -79.400049  
26          Chinatown, Grange Park, Kensington Market  43.653206 -79.400049  
27  CN Tower, Bathurst Quay, Island airport, Harbo...  43.628947 -79.394420  
28                    Stn A PO Boxes 25 The Esplanade  43.646435 -79.374846  
29             First Canadian Place, Underground city  43.64842

In [81]:
map_Toronto = folium.Map(location = [latitude,longitude], zoom_start= 10)

for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{},{}'.format(neighbourhood,borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Toronto)
 
map_Toronto

### Defining Foursquare Credentials and Version